In [ ]:
import importlib
import pandas as pd
import requests
import json
import functools
import glob
import base64
# import common
import multiprocessing.pool
import tldextract
import tqdm
import matplotlib
import threading
import itertools
import requests
import pickle


# load meta data 
file_dhcp = '../../../Validation Data/UNSW/scrape-results/dhcp_responses.csv'
file_endpoints = '../../../Validation Data/UNSW/scrape-results/endpoints.csv'
file_mdns= '../../../Validation Data/UNSW/scrape-results/mdns_responses.csv'
file_oui = '../../../Validation Data/UNSW/scrape-results/oui_friendly.csv'
file_ssdp = '../../../Validation Data/UNSW/scrape-results/ssdp_response.csv'
file_upnp = '../../../Validation Data/UNSW/scrape-results/upnp_responses.csv'
file_user_agent = '../../../Validation Data/UNSW/scrape-results/user_agent.csv'


file_dhcp = pd.read_csv(file_dhcp).drop(columns='Unnamed: 0')
file_endpoints = pd.read_csv(file_endpoints).drop(columns='Unnamed: 0').dropna().groupby('mac')['domain'].agg(lambda x: " + ".join(set(', '.join(x).split(', ')))).reset_index()
file_mdns = pd.read_csv(file_mdns).drop(columns='Unnamed: 0')
file_oui = pd.read_csv(file_oui).drop(columns='Unnamed: 0')
file_ssdp = pd.read_csv(file_ssdp).drop(columns='Unnamed: 0')
file_upnp = pd.read_csv(file_upnp).drop(columns='Unnamed: 0')
file_user_agent = pd.read_csv(file_user_agent).drop(columns='Unnamed: 0')
# Merge the dataframes one by one
merged_df = file_oui.merge(file_dhcp, on='mac', how='outer')
merged_df = merged_df.merge(file_mdns, on='mac', how='outer')
merged_df = merged_df.merge(file_ssdp, on='mac', how='outer')
merged_df = merged_df.merge(file_upnp, on='mac', how='outer')
merged_df = merged_df.merge(file_user_agent, on='mac', how='outer')
merged_df = merged_df.merge(file_endpoints, on='mac', how='outer')

print(merged_df.sample(5))

In [ ]:
import openai

OPENAI_API_KEY = 'sk-PLACE-YOUR-KEY-HERE'
openai.api_key = OPENAI_API_KEY
models = openai.Model.list()


def chat_completion(messages, max_tokens=20):
     return openai.Completion.create(
        engine='text-davinci-003',
        prompt= messages,
        max_tokens=max_tokens,
        temperature=0,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )

def ask_gpt_raw(messages, max_tokens=20):
    response = chat_completion(messages, max_tokens)
    return response.choices[0].text.translate(str.maketrans('', '', '\n\r\t'))

# get device identification from dhcp metadata 
get_dhcp_vendor = lambda s: ask_gpt_raw(f'I have an IoT device named "{s}". What is the company that makes this IoT device? Output the company\'s name only.', max_tokens=20)
get_dhcp_type = lambda s: ask_gpt_raw(f'I have an IoT device named "{s}". What type of IoT device is this? Output the name of the device type only.', max_tokens=20)

with multiprocessing.pool.ThreadPool(processes=10) as pool:
    merged_df['gpt_dhcp_vendor'] = pool.map(get_dhcp_vendor, merged_df['dhcp_response'].fillna(''))
    merged_df['gpt_dhcp_type'] = pool.map(get_dhcp_type, merged_df['dhcp_response'].fillna(''))


# get device identification from netdisco metadata 
get_netdisco_vendor = lambda s: ask_gpt_raw(f'I have an IoT device named "{s}". What is the company that makes this IoT device? Output the company\'s name only.', max_tokens=20)
get_netdisco_type = lambda s: ask_gpt_raw(f'I have an IoT device named "{s}". What type of IoT device is this? Output the name of the device type only.', max_tokens=20)

with multiprocessing.pool.ThreadPool(processes=5) as pool:
    merged_df['gpt_netdisco_vendor'] = pool.map(get_netdisco_vendor,
                                                merged_df['mdns_response'].fillna('') + ', ' +
                                                merged_df['ssdp_response'].fillna('') + ', ' +
                                                merged_df['upnp_response'].fillna(''))
    merged_df['gpt_netdisco_type'] = pool.map(get_netdisco_type,
                                              merged_df['mdns_response'].fillna('') + ', ' +
                                              merged_df['ssdp_response'].fillna('') + ', ' +
                                              merged_df['upnp_response'].fillna(''))

# get device identification from OUI and useragent  metadata 
get_oui_agent_vendor = lambda s: ask_gpt_raw(f'I have an IoT device with OUI name: "{s.split("::")[0]}", '
                                             f'and user agent: "{s.split("::")[1:]}"' 
                                             f'. What is the company that makes this IoT device? Output the company\'s name only.',
                                             max_tokens=20)
get_oui_agent_type = lambda s: ask_gpt_raw(f'I have an IoT device with OUI name: "{s.split("::")[0]}", '
                                           f'and user agent: "{s.split("::")[1:]}".'
                                           f' What type of IoT device is this? Output the name of the device type only.',
                                           max_tokens=20)

with multiprocessing.pool.ThreadPool(processes=5) as pool:
    merged_df['gpt_oui_agent_vendor'] = pool.map(get_oui_agent_vendor, 
                                                 merged_df['oui_friendly'].fillna('') + '::' +
                                                 merged_df['user_agent'].fillna(''))
    merged_df['gpt_oui_agent_type'] = pool.map(get_oui_agent_type,
                                               merged_df['oui_friendly'].fillna('') + '::' +
                                               merged_df['user_agent'].fillna(''))

# get device identification from only useragent  metadata 
get_user_agent_vendor = lambda s: ask_gpt_raw(f'I have an IoT device with  user agent: "{s}"'
                                              f'. What is the company that makes this IoT device? Output the company\'s name only.',
                                              max_tokens=10)
get_user_agent_type = lambda s: ask_gpt_raw(f'I have an IoT device with user agent: "{s}".'
                                            f' What type of IoT device is this? Output the name of the device type only.',
                                            max_tokens=10)


with multiprocessing.pool.ThreadPool(processes=5) as pool:
    merged_df['gpt_user_agent_vendor'] = pool.map(get_user_agent_vendor, merged_df['user_agent'].fillna(''))
    merged_df['gpt_user_agent_type'] = pool.map(get_user_agent_type, merged_df['user_agent'].fillna(''))
    
    
# merge all information into a single dataframe 
merged_df = merged_df[['mac', 'oui', 'oui_friendly', 'dhcp_response', 'mdns_response',
           'ssdp_response', 'upnp_response', 'user_agent', 'domain',
           'gpt_dhcp_vendor', 'gpt_netdisco_vendor', 'gpt_netdisco_type',
           'gpt_dhcp_type', 'gpt_oui_agent_vendor', 'gpt_oui_agent_type', 
           'gpt_user_agent_vendor', 'gpt_user_agent_type']]



## GPT Clean responses 

In [ ]:
gpt_two_feature_df = merged_df.copy(deep=True)

# Filter out crappy answers
def remove_bad_answer(s):
    if '___' in s:
        return ''

    if 'N/A' in s:
        return ''

    if s.endswith('.'):
        return s[0:-1]

    if 'unknown' in s.lower() or 'espressif' in s.lower() or 'ESP' in s:
        return ''

    if s == 'DLNA_DMR':
        return 'Streaming Device'

    return s

for col in gpt_two_feature_df.columns:
    if col.startswith('gpt_'):
        gpt_two_feature_df[col] = gpt_two_feature_df[col].fillna('')
        gpt_two_feature_df[col] = gpt_two_feature_df[col].apply(remove_bad_answer)

In [ ]:
stop_word_set = {
    'electronics', 'smart', 'inc', 'technology', 'espressif', 'assistant', 'technologies', 'corporation', 'ltd'
}

equivalent_companies = [
    {'lg', 'lg electronics'},
    {'nest', 'google'},
    {'myq', 'chamberlain'},
    {'xiaomi', 'yeelight'},
    {'dropcam', 'nest'}
]

def is_same_vendor(v1, v2):

    v1 = v1.lower().replace('.', '').replace('-', '')
    v2 = v2.lower().replace('.', '').replace('-', '')

    if v1 == '' or v2 == '':
        return ''

    # Same vendor if one is a substring of another
    if len(v1) >= 3 and v1 in v2 and v1 not in stop_word_set:
        return 'substring:' + v1
    if len(v2) >= 3 and v2 in v1 and v2 not in stop_word_set:
        return 'substring:' + v2

    # Same vendor if sharing at least one token that is not a stop word
    v1_tokens = set(v1.split())
    v2_tokens = set(v2.split())
    common_tokens = (v1_tokens & v2_tokens) - stop_word_set
    if common_tokens:
        return 'common_tokens:' + '+'.join(common_tokens)

    # Some of these tokens are substrings of each other
    for (t1, t2) in itertools.product(v1_tokens, v2_tokens):
        if len(t1) >= 3 and t1 in t2 and t1 not in stop_word_set:
            return 'common_token_substring:' + t1
        if len(t2) >= 3 and t2 in t1 and t2 not in stop_word_set:
            return 'common_token_substring:' + t2
        
    # TODO: [Added by Jakaria] Didn't consider the equivalent companies
    
    if {v1, v2} in equivalent_companies:
        return 'equivalent:'+v1

    return 'different'
    

In [ ]:
gpt_two_feature_consistency_df = gpt_two_feature_df.copy().fillna('')

columns_to_check = ['gpt_user_agent_vendor', 'gpt_dhcp_vendor', 'gpt_netdisco_vendor', 'oui_friendly', 'domains_friendly']

gpt_two_feature_consistency_df['domains_friendly'] = gpt_two_feature_consistency_df['domain'].fillna('').apply(
    lambda s: ' '.join(set([tldextract.extract(reg_domain).domain for reg_domain in s.split('+')]))
)

# Check across the columns for consistency
for (col1, col2) in itertools.combinations(columns_to_check, 2):
    new_col = f'consistency:{col1}:{col2}'
    gpt_two_feature_consistency_df[new_col] = gpt_two_feature_consistency_df.apply(
        lambda r: is_same_vendor(r[col1], r[col2]), axis=1
    )

# Consolidate
def consolidate_consistency(r):
    r = r.to_dict()

    common_term_set = set()
    for (k, v) in r.items():
        if k.startswith('consistency:') and ':' in v:
            common_term_set.add(v.split(':', 1)[1])

    # Some of the common terms are substrings of each other; merge these terms
    terms_to_remove = set()
    for (t1, t2) in itertools.permutations(common_term_set, 2):
        if len(t1) >=3 and t1 in t2:
            terms_to_remove.add(t2)

    common_term_set -= terms_to_remove

    # Merge equiv companies
    if common_term_set in equivalent_companies:
        common_term_set.pop()

    # If there are multiple terms and one of them is X (e.g. Spotify), remove X.
    if len(common_term_set) > 1:
        common_term_set -= {'spotify', 'google'}

    return '+'.join(common_term_set)

gpt_two_feature_consistency_df['consolidated_vendor'] = gpt_two_feature_consistency_df.apply(lambda r: consolidate_consistency(r), axis=1)

# Remove stop words and irrelevant results
def clean_consolidated_vendor(s):

    if 'hewlett' in s or 'packard' in s:
        return 'hp'

    if 'raspberry' in s:
        return 'raspberry-pi'

    if 'wemo' in s:
        return 'belkin'

    if s in ('ind', 'one', 'hon', 'shenzhen', 'electric', 'media', 'hom', 'ltd', 'digital', 'things', 'the', 'night', 'security'):
        return ''

    if s == 'free':
        return 'freebox'

    s = s.replace('electronics', '') \
        .replace('international inc', '') \
        .replace('inc', '') \
        .replace('corporation', '') \
        .replace('networks', '') \
        .replace('labs', '') \
        .replace('group', '') \
        .replace('llc', '') \
        .replace('foundation', '') \
        .replace('technology', '') \
        .replace('technologies', '') \
        .replace('ltd', '')

    return s.strip()

gpt_two_feature_consistency_df['consolidated_vendor'] = gpt_two_feature_consistency_df['consolidated_vendor'].apply(clean_consolidated_vendor)

In [ ]:
def get_clean_type(r):
    if r['gpt_user_agent_type']:
        if (r['gpt_dhcp_type'] == r['gpt_user_agent_type']) or (r['gpt_netdisco_type'] == r['gpt_user_agent_type']):
            return r['gpt_user_agent_type']

    if r['gpt_dhcp_type']:
        if (r['gpt_dhcp_type'] == r['gpt_netdisco_type']) or (r['gpt_dhcp_type'] == r['gpt_user_agent_vendor']):
            return r['gpt_dhcp_type']

    if r['gpt_netdisco_type']:
        if (r['gpt_dhcp_type'] == r['gpt_netdisco_type']) or (r['gpt_netdisco_type'] == r['gpt_user_agent_vendor']):
            return r['gpt_netdisco_type']

    if r['gpt_user_agent_type'] and r['gpt_user_agent_vendor'] != '':
        return r['gpt_user_agent_type']
    if r['gpt_dhcp_type'] and r['gpt_dhcp_vendor'] != '':
        return r['gpt_dhcp_type']
    if r['gpt_netdisco_type'] and r['gpt_netdisco_vendor'] != '':
        return r['gpt_netdisco_type']
    return ''


gpt_clean_df = gpt_two_feature_consistency_df.copy()
gpt_clean_df['gpt_clean_vendor'] = gpt_clean_df['consolidated_vendor']
gpt_clean_df['gpt_clean_type'] = gpt_clean_df.apply(get_clean_type, axis=1)


file_ground_truth = '../../../Validation Data/UNSW/List_of_Devices.csv'
ground_truth = pd.read_csv(file_ground_truth)

gpt_clean_df = gpt_clean_df.merge(ground_truth, on='mac', how='outer')

# todo save result in file 
# file_to_save = '../../../Validation Data/UNSW/gpt-clean-updated.csv'
# gpt_clean_df.to_csv(file_to_save, index=False)